In [17]:
%run lm_ode.ipynb

In [2]:
import matplotlib.pyplot as plt

# Model 2: Pyrolytic dehydrogenation model
Pyrolytic dehydrogenation of benzene to diphenyl and triphenyl
$$2C_2H_6\longleftrightarrow C_{12}H_{10}+H_2$$
$$C_6H_6+C_{12}H_{10}\longleftrightarrow C_{10}H_{14}+H_2$$
with differential equations model
\begin{align*}
\frac{dy_1}{dt}=&-r_1-r_2;\hspace{10mm}y_1(0)=1\\
\frac{dy_2}{dt}=&\frac{r_1}{2}-r_2;\hspace{10mm}y_2(0)=0\\
r_1=&k_1\left(y_1^2-y_2\frac{2-2y_1-y_2}{3K_1}\right)\\
r_2=&k_2\left(y_1y_2-\frac{(1-y_1-2y_2)(2-2y_1-y_2)}{9K_2}\right)
\end{align*}
with $K_1=0.242$ and $K_2=0.428$

In [3]:
def model2(y,k):
    K1 = 0.242
    K2 = 0.428
    r1 = k[0]*(y[0]**2-y[1]*(2-2*y[0]-y[1])/(3*K1))
    r2 = k[1]*(y[0]*y[1]-(1-y[0]-2*y[1])*(2-2*y[0]-y[1])/(9*K2))
    dydt = np.empty(2)
    dydt[0] = -r1-r2
    dydt[1] = r1/2-r2
    return dydt

## Scaling for parameter estimation
Let $u_1=\frac{y_1}{M_1}$ and $u_2=\frac{y_2}{M_2}$,
\begin{align*}
\frac{M_1}{t_c}\frac{du_1}{d\bar{t}}=&-k_1\left(M_1^2u_1^2-M_2u_2\frac{2-2M_1u_1-M_2u_2}{3K_1}\right)\\
&-k_2\left(M_1M_2u_1u_2-\frac{(1-M_1u_1-2M_2u_2)(2-2M_1u_1-M_2u_2)}{9K_2}\right)\\
=&-k_1M_1^2\left(u_1^2-\frac{M_2}{M_1}u_2\frac{\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2}{3K_1}\right)\\
&-k_2M_1^2\left(\frac{M_2}{M_1}u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\frac{M_2}{M_1}u_2)(\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2)}{9K_2}\right)\\
\frac{du_1}{d\bar{t}}=&-k_1M_1t_c\left(u_1^2-\frac{M_2}{M_1}u_2\frac{\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2}{3K_1}\right)\\
&-k_2M_1t_c\left(\frac{M_2}{M_1}u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\frac{M_2}{M_1}u_2)(\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2)}{9K_2}\right)
\end{align*}

\begin{align*}
\frac{M_2}{t_c}\frac{du_2}{d\bar{t}}=&\frac{k_1}{2}\left(M_1^2u_1^2-M_2u_2\frac{2-2M_1u_1-M_2u_2}{3K_1}\right)\\
&-k_2\left(M_1M_2u_1u_2-\frac{(1-M_1u_1-2M_2u_2)(2-2M_1u_1-M_2u_2)}{9K_2}\right)\\
=&\frac{k_1M_1^2}{2}\left(u_1^2-\frac{M_2}{M_1}u_2\frac{\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2}{3K_1}\right)\\
&-k_2M_1^2\left(\frac{M_2}{M_1}u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\frac{M_2}{M_1}u_2)(\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2)}{9K_2}\right)\\
\frac{du_2}{d\bar{t}}=&\frac{k_1M_1t_c}{2}\frac{M_1}{M_2}\left(u_1^2-\frac{M_2}{M_1}u_2\frac{\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2}{3K_1}\right)\\
&-k_2M_1t_c\frac{M_1}{M_2}\left(\frac{M_2}{M_1}u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\frac{M_2}{M_1}u_2)(\frac{2}{M_1}-2u_1-\frac{M_2}{M_1}u_2)}{9K_2}\right)
\end{align*}

Let $\bar{k_1}=k_1M_1t_c$, $\bar{k_2}=k_2M_1t_c$ and $\frac{M_2}{M_1}=\gamma$,
\begin{align*}
\frac{du_1}{d\bar{t}}=&-\bar{k}_1\left(u_1^2-\gamma u_2\frac{\frac{2}{M_1}-2u_1-\gamma u_2}{3K_1}\right)\\
&-\bar{k}_2\left(\gamma u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\gamma u_2)(\frac{2}{M_1}-2u_1-\gamma u_2)}{9K_2}\right)
\end{align*}
\begin{align*}
\frac{du_2}{d\bar{t}}=&\frac{\bar{k}_1}{2\gamma}\left(u_1^2-\gamma u_2\frac{\frac{2}{M_1}-2u_1-\gamma u_2}{3K_1}\right)\\
&-\frac{\bar{k}_2}{\gamma}\left(\gamma u_1u_2-\frac{(\frac{1}{M_1}-u_1-2\gamma u_2)(\frac{2}{M_1}-2u_1-\gamma u_2)}{9K_2}\right)
\end{align*}

In [4]:
def model2_scale(u,kbar,M1,M2):
    K1 = 0.242
    K2 = 0.428
    gamma = M2/M1
    r1 = u[0]**2-gamma*u[1]*(2/M1-2*u[0]-gamma*u[1])/(3*K1)
    r2 = gamma*u[0]*u[1]-(1/M1-u[0]-2*gamma*u[1])*(2/M1-2*u[0]-gamma*u[1])/(9*K2)
    dudt = np.empty(2)
    dudt[0] = -kbar[0]*r1-kbar[1]*r2
    dudt[1] = kbar[0]*r1/(2*gamma)-kbar[1]*r2/gamma
    return dudt

## Integration with estimated paramters

In [12]:
t = np.array([0,5.63,11.32,16.97,22.62,34.00,39.70,45.20,169.7],dtype='f')*1e-4
yhat = np.array([[1,0.828,0.704,0.622,0.565,0.499,0.482,0.470,0.443],
                   [0,0.0737,0.1130,0.1322,0.1400,0.1468,0.1477,0.1477,0.1476]],dtype='f')
y0 = yhat[:,0]
k = np.array([354.61,400.23])
k0 = np.array([500.0,500.0])
Q = np.eye(2)

In [10]:
tc = t[-1]
M1 = max(yhat[0,:])
M2 = max(yhat[1,:])
tbar = t/tc
uhat = np.diag([1/M1,1/M2])@yhat
u0 = uhat[:,0]
kbar = k*M1*tc

In [7]:
def model2_scaled(u,kbar):
    return model2_scale(u,kbar,M1,M2)

## Parameter estimation with LM mothod
### Test with given parameters

### Initial guess with $k=[500,500]$

In [13]:
est = gnlm(model2,yhat,Q,k,t,[1,1e-8,1e-8,100])

First guess was correct!
Iter | Obj func | step size | gradient |   mu   |   rho
    0|3.5252e-06|   Not cal |  Not cal | 1.3e-06| Not cal


In [14]:
est

[array([354.61, 400.23]),
 array([[1.        , 0.82833154, 0.70540729, 0.62146649, 0.56432773,
         0.49898196, 0.48110363, 0.46933863, 0.44329932],
        [0.        , 0.07379238, 0.11221172, 0.131312  , 0.14069535,
         0.14727443, 0.14814507, 0.14844049, 0.14773841]]),
 [0, 'g']]

In [18]:
est = gnlm(model2,yhat,Q,k0,t,[1e-3,1e-8,1e-8,100])

Iter | Obj func | step size | gradient |   mu   |   rho
    0|8.9248e-03|   Not cal |  Not cal | 5.0e-10| Not cal
    1|9.4418e-04|  2.168e+02|  5.58e-05| 2.2e-10| 9.1e-01
    2|7.5013e-06|  3.803e+01|  3.20e-06| 7.2e-11| 1.0e+00
    3|3.5253e-06|  2.587e+00|  2.46e-08| 2.4e-11| 1.0e+00
    4|3.5253e-06|  2.335e-02|  2.46e-08| 4.8e-11| 1.0e+00
    5|3.5253e-06|  2.335e-02|  1.98e-11| 5.2e-11| 2.8e-01


In [16]:
est

[array([354.60676151, 400.21730922]),
 array([[1.        , 0.82833301, 0.70540951, 0.62146886, 0.56432996,
         0.49898358, 0.48110495, 0.46933968, 0.44329934],
        [0.        , 0.07379208, 0.11221161, 0.13131226, 0.14069586,
         0.14727506, 0.14814565, 0.14844101, 0.14773841]]),
 [5, 'g']]

In [22]:
ktry = np.array([5000.0,5000.0])

In [25]:
est = gnlm(model2,yhat,Q,ktry,t,[1e2,1e-8,1e-8,100])

Iter | Obj func | step size | gradient |   mu   |   rho
    0|1.3640e-01|   Not cal |  Not cal | 5.1e-10| Not cal
    1|1.3191e-01|  1.773e+03|  5.65e-06| 1.7e-10| 2.8e+00
    2|1.3191e-01|  1.468e+04|  5.65e-06| 3.4e-10| 2.8e+00
    3|1.3191e-01|  1.019e+04|  5.65e-06| 1.4e-09| 2.8e+00
    4|1.3191e-01|  3.604e+03|  5.65e-06| 1.1e-08| 2.8e+00
    5|1.2800e-01|  5.125e+02|  9.99e-06| 3.6e-09| 1.4e+00
    6|1.4924e-02|  2.330e+03|  9.99e-05| 1.2e-09| 5.3e+00
    7|6.9698e-03|  3.436e+02|  6.30e-05| 5.9e-10| 9.0e-01
    8|5.5540e-03|  7.338e+02|  3.81e-06| 2.0e-10| 1.1e+00
    9|1.4363e-03|  2.445e+03|  1.01e-05| 6.5e-11| 3.0e+00
   10|1.4363e-03|  6.750e+02|  1.01e-05| 1.3e-10| 3.0e+00
   11|1.4363e-03|  6.672e+02|  1.01e-05| 5.2e-10| 3.0e+00
   12|1.4363e-03|  6.241e+02|  1.01e-05| 4.2e-09| 3.0e+00
   13|5.1999e-05|  3.891e+02|  4.84e-06| 1.4e-09| 1.3e+00
   14|3.5934e-06|  4.788e+01|  2.38e-07| 4.6e-10| 1.0e+00
   15|3.5254e-06|  1.446e+00|  6.43e-10| 1.5e-10| 1.0e+00


In [26]:
est

[array([354.60683035, 400.20282132]),
 array([[1.        , 0.82833321, 0.70541003, 0.62146958, 0.56433075,
         0.49898428, 0.48110554, 0.46934018, 0.4432997 ],
        [0.        , 0.07379237, 0.11221237, 0.13131337, 0.14069712,
         0.14727624, 0.14814668, 0.14844188, 0.14773845]]),
 [15, 'g']]